In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'
file_path = '../bar_movement/data/'

In [3]:
currency_pair = 'Eur_Usd'
rounding = 3 if 'Jpy' in currency_pair else 5
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000
year_range = '2021-2022'
fractal_window = 15
divider = 100 if 'Jpy' in currency_pair else 10000

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{year_range}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{year_range}.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [5]:
def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
    ma_down = down.ewm(com=periods - 1, adjust=True,
                       min_periods=periods).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))

    return rsi

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def smma(closes, length):
    smma = []

    for i in range(len(closes)):
        if i < length:
            smma.append(closes.iloc[:i + 1,].rolling(length).mean().iloc[-1,])

        else:
            smma.append((smma[i - 1] * (length - 1) + closes[i]) / length)

    return pd.Series(smma)

def fractal(lows, highs, window=20):
    assert len(lows) == len(highs)

    fractal_period = 2 * window + 1

    is_support = lows.rolling(fractal_period, center=True).apply(lambda x: x[window] == min(x), raw=True)
    is_resistance = highs.rolling(fractal_period, center=True).apply(lambda x: x[window] == max(x), raw=True)
    
    is_support_indices = pd.Series(is_support.index[is_support == 1.0])
    is_resistance_indices = pd.Series(is_resistance.index[is_resistance == 1.0])

    support_fractal_vals = lows[is_support_indices].reindex(lows.index).ffill()
    resistance_fractal_vals = highs[is_resistance_indices].reindex(highs.index).ffill()

    return support_fractal_vals, resistance_fractal_vals

In [6]:
df['rsi'] = rsi(df['Mid_Close'])
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['support_fractal'], df['resistance_fractal'] = fractal(df['Mid_Low'], df['Mid_High'], fractal_window)
df['support_fractal'], df['resistance_fractal'] = df['support_fractal'].shift(fractal_window), df['resistance_fractal'].shift(fractal_window)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['smma200'] = smma(df['Mid_Close'], 200)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long['support_fractal'], df_long['resistance_fractal'] = fractal(df_long['Mid_Low'], df_long['Mid_High'], fractal_window)
df_long['support_fractal'], df_long['resistance_fractal'] = df_long['support_fractal'].shift(fractal_window), df_long['resistance_fractal'].shift(fractal_window)
df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

In [7]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,rsi_sma,atr,lower_atr_band,upper_atr_band,macd,macdsignal,support_fractal,resistance_fractal,ema200,smma200
0,2021-04-01 22:35:00,1.17785,1.17788,1.17783,1.17784,1.17801,1.17805,1.17799,1.17800,1.17793,...,56.766621,0.000149,1.177472,1.178368,0.000129,0.000126,1.17668,1.178,1.176456,1.175557
1,2021-04-01 22:40:00,1.17785,1.17787,1.17784,1.17787,1.17802,1.17802,1.17800,1.17800,1.17794,...,57.124495,0.000130,1.177550,1.178330,0.000125,0.000126,1.17668,1.178,1.176473,1.175569
2,2021-04-01 22:45:00,1.17788,1.17804,1.17784,1.17800,1.17802,1.17817,1.17799,1.17815,1.17795,...,57.537448,0.000141,1.177658,1.178502,0.000132,0.000127,1.17668,1.178,1.176491,1.175582
3,2021-04-01 22:50:00,1.17801,1.17819,1.17798,1.17810,1.17817,1.17834,1.17813,1.17823,1.17809,...,57.935115,0.000146,1.177723,1.178597,0.000142,0.000130,1.17668,1.178,1.176511,1.175594
4,2021-04-01 22:55:00,1.17807,1.17807,1.17792,1.17795,1.17822,1.17822,1.17807,1.17811,1.17814,...,58.183663,0.000139,1.177612,1.178448,0.000138,0.000132,1.17668,1.178,1.176528,1.175607


In [8]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,...,rsi_sma,atr,lower_atr_band,upper_atr_band,macd,macdsignal,support_fractal,resistance_fractal,ema200,smma200
74579,2022-04-01 05:35:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,...,49.066137,0.000272,1.106024,1.107656,0.000018,-0.000020,1.10614,1.10732,1.107953,1.109396
74580,2022-04-01 05:40:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,...,49.186113,0.000299,1.106382,1.108178,0.000065,-0.000003,1.10614,1.10732,1.107946,1.109386
74581,2022-04-01 05:45:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,...,49.253825,0.000289,1.106254,1.107986,0.000087,0.000015,1.10614,1.10732,1.107938,1.109374
74582,2022-04-01 05:50:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,...,49.376518,0.000294,1.106459,1.108221,0.000122,0.000037,1.10628,1.10732,1.107932,1.109364
74583,2022-04-01 05:55:00,1.10727,1.10737,1.10667,1.10673,1.10742,1.10754,1.10683,1.10690,1.10734,...,49.231204,0.000331,1.105826,1.107814,0.000106,0.000050,1.10628,1.10732,1.107921,1.109351


In [9]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]

In [14]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def is_in_smooth_trend(curr_idx, trend_type, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant):
    supports, resistances = [], []
    curr_support, curr_resistance = None, None

    for j in range(curr_idx - 1, -1, -1):
        mid_open, mid_high, mid_low, mid_close, ma, support, resistance = df.loc[df.index[j], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close', ma_key, 'support_fractal', 'resistance_fractal']]
        curr_support = support if curr_support is None else curr_support
        curr_resistance = resistance if curr_resistance is None else curr_resistance

        # Check for invalidation
        if trend_type == 'up':
            if (touch_ma_invalidates and mid_low <= ma) or min([mid_open, mid_close]) <= ma:
                return False
            
        else:
            if (touch_ma_invalidates and mid_high >= ma) or max([mid_open, mid_close]) >= ma:
                return False

        # Update supports and resistances, if we see new values
        if mid_low == curr_support:
            supports.append(curr_support)
            curr_support = support
        
        if mid_high == resistance:
            resistances.append(curr_resistance)
            curr_resistance = resistance

        # Check for success cases
        if (minor_one_less and trend_type == 'up' and len(resistances) >= n_smooth_fractals and len(supports) >= n_smooth_fractals - 1) or \
           (minor_one_less and trend_type == 'down' and len(supports) >= n_smooth_fractals and len(resistances) >= n_smooth_fractals - 1) or \
           (len(supports) >= n_smooth_fractals and len(resistances) >= n_smooth_fractals):
            
            passed_recent_level = mid_close > resistances[0] if price_passed_recent_level and trend_type == 'up' else (mid_close < supports[0] if price_passed_recent_level and trend_type == 'down' else True)
            significant_level = resistances[0] > resistances[1] if recent_level_significant and trend_type == 'up' else (supports[0] < supports[1] if recent_level_significant and trend_type == 'down' else True)

            if passed_recent_level and significant_level:
                return True

    return False

def broke_level(curr_idx, curr_close, trend_type, n_bars_since_break, passed_break, within_atr_multiplier):
    for j in range(curr_idx - 1, max(curr_idx - 1 - n_bars_since_break, -1), -1):
        curr_date = df.loc[df.index[j], 'Date']
        curr_long = df_long.loc[df_long['Date'] <= curr_date]

        if len(curr_long) < 2 or len(df_long.loc[df_long.Date >= curr_date]) == 0:
            return False

        support, resistance = curr_long.loc[curr_long.index[-2], ['support_fractal', 'resistance_fractal']]
        mid_close, atr = df.loc[df.index[j], ['Mid_Close', 'atr']]
        min_pips_to_level = atr * within_atr_multiplier
        pips_to_level = abs(mid_close - support) if trend_type == 'down' else abs(mid_close - resistance)
        close_enough_to_level = pips_to_level <= min_pips_to_level

        if close_enough_to_level:
            if (trend_type == 'up' and mid_close > resistance) and ((passed_break and curr_close > resistance) or (not passed_break and curr_close < resistance)):
                return True

            elif (trend_type == 'down' and mid_close < support) and ((passed_break and curr_close < support) or (not passed_break and curr_close > support)):
                return True

    return False

def was_retest_pattern(curr_idx, trend_type, n_bars_for_each):
    required_len = n_bars_for_each * 2
    mid_opens = list(df.loc[df.index[max(curr_idx - required_len, 0): curr_idx], 'Mid_Open'])
    mid_closes = list(df.loc[df.index[max(curr_idx - required_len, 0): curr_idx], 'Mid_Close'])

    assert len(mid_opens) == len(mid_closes)

    if len(mid_opens) < required_len:
        return False

    for j in range(0, n_bars_for_each):
        if (trend_type == 'up' and mid_opens[j] < mid_closes[j]) or (trend_type == 'down' and mid_opens[j] > mid_closes[j]):
            return False

    for j in range(n_bars_for_each, required_len):
        if (trend_type == 'up' and mid_opens[j] > mid_closes[j]) or (trend_type == 'down' and mid_opens[j] < mid_closes[j]):
            return False

    return True

def run_simulation(spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, risk_reward_ratio, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant, n_bars_since_break, passed_break, within_atr_multiplier, n_bars_for_each):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts, trade_dates = [], [], [], []
    trade = None

    for i in range(2, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        if trade is None:
            atr, lower_atr_band, upper_atr_band, rsi, rsi_sma, ma, mid_close = df.loc[df.index[i - 1], ['atr', 'lower_atr_band', 'upper_atr_band', 'rsi', 'rsi_sma', ma_key, 'Mid_Close']]
            rsi_buy = rsi > rsi_sma if use_rsi_sma else True
            rsi_sell = rsi < rsi_sma if use_rsi_sma else True
            trend = 'up' if mid_close > ma else ('down' if mid_close < ma else 'none')

            if rsi_buy and trend == 'up':
                if broke_level(i, mid_close, trend, n_bars_since_break, passed_break, within_atr_multiplier) and is_in_smooth_trend(i, trend, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant) and was_retest_pattern(i, trend, n_bars_for_each):
                    open_price = float(curr_ao)
                    sl_pips = atr if sl_method == 'atr' else abs(open_price - lower_atr_band)
                    stop_loss = round(open_price - (sl_pips * sl_multiplier), rounding)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if spread <= curr_pips_to_risk * spread_cutoff:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)
                            n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                            trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                    'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                            
                            pips_risked.append(curr_pips_to_risk)
                            n_buys += 1
                            trade_dates.append(curr_date)

            elif rsi_sell and trend == 'down':
                if broke_level(i, mid_close, trend, n_bars_since_break, passed_break, within_atr_multiplier) and is_in_smooth_trend(i, trend, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant) and was_retest_pattern(i, trend, n_bars_for_each):
                    open_price = float(curr_bo)
                    sl_pips = atr if sl_method == 'atr' else abs(open_price - upper_atr_band)
                    stop_loss = round(open_price + (sl_pips * sl_multiplier), rounding)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if spread <= curr_pips_to_risk * spread_cutoff:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)
                            n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                            trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                    'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                            
                            pips_risked.append(curr_pips_to_risk)
                            n_sells += 1
                            trade_dates.append(curr_date)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, trade_dates

In [16]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------

# spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, risk_reward_ratio, macd_type, macd_atr_threshold_mult, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant

spread_cutoffs = [0.1]
sl_methods = ['atr', 'atr_bands']
# sl_methods = ['atr_bands']
# sl_multipliers = [1.0, 2.0, 3.0]
sl_multipliers = [2.0, 3.0]
use_trailing_sl_vals = [True]
risk_reward_ratios = [0.0]
use_rsi_sma_vals = [False]
ma_keys = ['ema200', 'smma200']
# ma_keys = ['ema200']
touch_ma_invalidates_vals = [True, False]
# touch_ma_invalidates_vals = [False]
n_smooth_fractals_vals = [2, 3]
minor_one_less_vals = [True, False]
price_passed_recent_level_vals = [True, False]
# price_passed_recent_level_vals = [False]
recent_level_significant_vals = [True, False]
# recent_level_significant_vals = [True]
n_bars_since_break_vals = [12, 24]
passed_break_vals = [True, False]
within_atr_multipliers = [1.0, 1.5, 2.0, 3.0]
n_bars_for_each_vals = [2]

all_combos = []

for spread_cutoff in spread_cutoffs:
    for sl_method in sl_methods:
        for sl_multiplier in sl_multipliers:
            for use_rsi_sma in use_rsi_sma_vals:
                for ma_key in ma_keys:
                    for touch_ma_invalidates in touch_ma_invalidates_vals:
                        for n_smooth_fractals in n_smooth_fractals_vals:
                            for minor_one_less in minor_one_less_vals:
                                for price_passed_recent_level in price_passed_recent_level_vals:
                                    for recent_level_significant in recent_level_significant_vals:
                                        for n_bars_since_break in n_bars_since_break_vals:
                                            for passed_break in passed_break_vals:
                                                for within_atr_multiplier in within_atr_multipliers:
                                                    for n_bars_for_each in n_bars_for_each_vals:
                                                        for use_trailing_sl in use_trailing_sl_vals:
                                                            for risk_reward_ratio in risk_reward_ratios:
                                                                rr_ratio = 0.0 if use_trailing_sl else risk_reward_ratio
                                                                all_combos.append((spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, rr_ratio, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant, n_bars_since_break, passed_break, within_atr_multiplier, n_bars_for_each))

                                                                if use_trailing_sl:
                                                                    break


percentage_to_try = 0.25
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_spread_cutoff, best_sl_method, best_sl_multiplier, best_use_trailing_sl, best_risk_reward_ratio, best_use_rsi_sma, best_ma_key, best_touch_ma_invalidates, best_n_smooth_fractals, best_minor_one_less, best_price_passed_recent_level, best_recent_level_significant, best_n_bars_since_break, best_passed_break, best_within_atr_multiplier, best_n_bars_for_each = None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, risk_reward_ratio, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant, n_bars_since_break, passed_break, within_atr_multiplier, n_bars_for_each in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, trade_dates = run_simulation(spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, risk_reward_ratio, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant, n_bars_since_break, passed_break, within_atr_multiplier, n_bars_for_each)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))
    print(trade_dates)

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'spread_cutoff': spread_cutoff, 'sl_method': sl_method, 'sl_multiplier': sl_multiplier, 'use_trailing_sl': use_trailing_sl, 'risk_reward_ratio': risk_reward_ratio, 'use_rsi_sma': use_rsi_sma, 'ma_key': ma_key, 'touch_ma_invalidates': touch_ma_invalidates, 'n_smooth_fractals': n_smooth_fractals, 'minor_one_less': minor_one_less, 'price_passed_recent_level': price_passed_recent_level, 'recent_level_significant': recent_level_significant, 'n_bars_since_break': n_bars_since_break, 'passed_break': passed_break, 'within_atr_multiplier': within_atr_multiplier, 'n_bars_for_each': n_bars_for_each})

    if total_profit > best_reward:
        best_reward = total_profit
        best_spread_cutoff, best_sl_method, best_sl_multiplier, best_use_trailing_sl, best_risk_reward_ratio, best_use_rsi_sma, best_ma_key, best_touch_ma_invalidates, best_n_smooth_fractals, best_minor_one_less, best_price_passed_recent_level, best_recent_level_significant, best_n_bars_since_break, best_passed_break, best_within_atr_multiplier, best_n_bars_for_each = spread_cutoff, sl_method, sl_multiplier, use_trailing_sl, risk_reward_ratio, use_rsi_sma, ma_key, touch_ma_invalidates, n_smooth_fractals, minor_one_less, price_passed_recent_level, recent_level_significant, n_bars_since_break, passed_break, within_atr_multiplier, n_bars_for_each


    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 1024

99.99935999999978 -2.7777600000000002 97.22159999999978
Num buys: 1
Num sells: 0
Num trades: 1
Num wins: 1
Num losses: 0
Win streak: 1
Loss streak: 0
Avg pips risked: 0.0014399999999998858
Avg win amount: 99.99935999999978
Min win amount: 99.99935999999978
Max win amount: 99.99935999999978
[Timestamp('2022-03-30 12:40:00')]
Remaining runs: 1023
Best reward so far: 97.22159999999978

0 0 0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
[]
Remaining runs: 1022
Best reward so far: 97.22159999999978

249.99907999999968 -8.97302 241.0260599999997
Num buys: 5
Num sells: 2
Num trades: 7
Num wins: 3
Num losses: 4
Win streak: 2
Loss streak: 4
Avg pips risked: 0.0029400000000000376
Avg win amount: 149.9972566666666
Min win amount: 49.99860000000276
Max win amount: 349.993279999999
Avg loss amount: -49.99817250000004
Min loss amount: -49.99949999999991
Max loss amount: -49.99582999999992
[Timestamp('2021-05-18 06:45:00'), Timestamp('2

KeyboardInterrupt: 

In [13]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread cutoff: ' + str(best_spread_cutoff))
print('Best sl method: ' + str(best_sl_method))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best use trailing sl val: ' + str(best_use_trailing_sl))
print('Best risk reward ratio: ' + str(best_risk_reward_ratio))
print('Best use rsi sma val: ' + str(best_use_rsi_sma))
print('Best ma key: ' + str(best_ma_key))
print('Best touch ma invalidates val: ' + str(best_touch_ma_invalidates))
print('Best n smooth fractals: ' + str(best_n_smooth_fractals))
print('Best minor one less val: ' + str(best_minor_one_less))
print('Best price passed recent level val: ' + str(best_price_passed_recent_level))
print('Best recent level significant val: ' + str(best_recent_level_significant))
print('Best n bars since break: ' + str(best_n_bars_since_break))
print('Best passed break val: ' + str(best_passed_break))
print('Best within atr multiplier: ' + str(best_within_atr_multiplier))
print('Best n bars for each: ' + str(best_n_bars_for_each))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 425.8357800000001
Best spread cutoff: 0.1
Best sl method: atr_bands
Best sl multiplier: 3.0
Best use trailing sl val: True
Best risk reward ratio: 0.0
Best macd type: impulse_macd
Best macd atr threshold: 0.5
Best use rsi sma val: False
Best ma key: smma200
Best touch ma invalidates val: True
Best n smooth fractals: 2
Best minor one less val: False
Best price passed recent level val: True
Best recent level significant val: False
Best n bars since break: 12
Best passed break val: True
-----------------------
Top results:
{'reward': 374, 'spread_cutoff': 0.1, 'sl_method': 'atr_bands', 'sl_multiplier': 3.0, 'use_trailing_sl': True, 'risk_reward_ratio': 0.0, 'macd_type': 'impulse_macd', 'macd_atr_threshold_mult': 1.0, 'use_rsi_sma': False, 'ma_key': 'smma200', 'touch_ma_invalidates': True, 'n_smooth_fractals': 2, 'minor_one_less': True, 'price_passed_recent_level': True, 'recent_level_significant': False, 'n_bars_since_break': 12, 'passe

In [16]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread cutoff: ' + str(best_spread_cutoff))
print('Best sl method: ' + str(best_sl_method))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('Best use trailing sl val: ' + str(best_use_trailing_sl))
print('Best risk reward ratio: ' + str(best_risk_reward_ratio))
print('Best macd type: ' + str(best_macd_type))
print('Best macd atr threshold: ' + str(best_macd_atr_threshold_mult))
print('Best use rsi sma val: ' + str(best_use_rsi_sma))
print('Best ma key: ' + str(best_ma_key))
print('Best touch ma invalidates val: ' + str(best_touch_ma_invalidates))
print('Best n smooth fractals: ' + str(best_n_smooth_fractals))
print('Best minor one less val: ' + str(best_minor_one_less))
print('Best price passed recent level val: ' + str(best_price_passed_recent_level))
print('Best recent level significant val: ' + str(best_recent_level_significant))
print('Best n bars since break: ' + str(best_n_bars_since_break))
print('Best passed break val: ' + str(best_passed_break))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 764.9758099999966
Best spread cutoff: 0.1
Best sl method: atr_bands
Best sl multiplier: 2.0
Best use trailing sl val: True
Best risk reward ratio: 0.0
Best macd type: n_macd
Best macd atr threshold: 0.5
Best use rsi sma val: False
Best ma key: smma200
Best touch ma invalidates val: False
Best n smooth fractals: 2
Best minor one less val: True
Best price passed recent level val: False
Best recent level significant val: False
Best n bars since break: 24
Best passed break val: False
-----------------------
Top results:
{'reward': 457, 'spread_cutoff': 0.1, 'sl_method': 'atr_bands', 'sl_multiplier': 2.0, 'use_trailing_sl': True, 'risk_reward_ratio': 0.0, 'macd_type': 'n_macd', 'macd_atr_threshold_mult': 0.0, 'use_rsi_sma': False, 'ma_key': 'smma200', 'touch_ma_invalidates': True, 'n_smooth_fractals': 2, 'minor_one_less': True, 'price_passed_recent_level': False, 'recent_level_significant': True, 'n_bars_since_break': 24, 'passed_break': 